<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/mfml1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [190]:
#Create matrix
def create_random_matrix(m,n,generate_float=False):
    if generate_float == True:
        A = np.random.ranf(size=(m, n))
    else:
        A = np.random.randint(low=0,high=100,size=(m, n))
        A=A.astype(float)
    return A

def augment_matrix(A,b):
    b=b.reshape(b.shape[0],1)
    return np.append(A,b,axis=1)


In [35]:
#Elementary row operations
def const_multiple(A,i,c):
    A[i]*=c
def exchange_rows(A,i,j):
    A[[i,j]] = A[[j,i]]
def add_const_row_multiple(A,i,j,c):
    A[i]+=A[j]*c


In [254]:
def find_pivot_row(A, curr_pivot_row, col):
    rows = A.shape[0]
    for row in range(curr_pivot_row, rows):
        if A[row, col] != 0.0:
            return row
    return None

def convert_pivot_value_to_one(A,pivot_row,col):
    pivot_row_value=A[pivot_row,col]
    A[pivot_row] /=pivot_row_value


def convert_pivot_row_column_values_to_zero_above(A,pivot_row,col):
    rows = A.shape[0]
    for row in range(0,pivot_row):
        const_multiplier=A[row][col]/A[pivot_row][col]
        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]

def convert_pivot_row_column_values_to_zero_below(A,pivot_row,col):
    rows = A.shape[0]
    for row in range(pivot_row+1, rows):
        const_multiplier=A[row][col]/A[pivot_row][col]

        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]

def calculate_row_echelon_form(A,rref=False):


    pivot_row_cols=[]
    pivot_cols=[]
    non_pivot_cols=[]
    rows=A.shape[0]
    cols=A.shape[1]

    def calculate_pivot_and_non_pivot_cols():
        all_cols=np.array(range(cols))
        pivot_cols_list=[]
        for pivot_row_col in pivot_row_cols:
            pivot_cols_list.append(pivot_row_col[1])
        pivot_cols=np.array(pivot_cols_list)
        non_pivot_cols=np.setdiff1d(all_cols,pivot_cols)
        return pivot_cols,non_pivot_cols


    curr_pivot_row=0
    for col in range(cols):
        nonzero_row = find_pivot_row(A, curr_pivot_row, col)
        if nonzero_row is not None:
            exchange_rows(A,curr_pivot_row,nonzero_row)
            #print('pivot row={} column={} '.format(curr_pivot_row,col))
            pivot_row_cols.append((curr_pivot_row,col))
            if rref == True:
                convert_pivot_value_to_one(A,curr_pivot_row,col)
            convert_pivot_row_column_values_to_zero_below(A,curr_pivot_row,col)
            if rref == True:
                convert_pivot_row_column_values_to_zero_above(A,curr_pivot_row,col)
            curr_pivot_row+=1
    pivot_cols,non_pivot_cols=calculate_pivot_and_non_pivot_cols()
    return pivot_row_cols,pivot_cols,non_pivot_cols

def calculate_generic_solution(A,b):
    if A.shape[0] != b.shape[0]:
        print("Illegal values for A and b")
        return
    A_aug=augment_matrix(A,b)
    pivot_row_cols,pivot_cols , non_pivot_cols = calculate_row_echelon_form(A,rref=True)
    x_particular=np.zeros(A.shape[1],dtype=float)
    x_nullspace_solutions=[]
    last_col=A_aug.shape[1]-1
    for pivot_row_col in pivot_row_cols:
        x_particular[pivot_row_col[1]]=A_aug[ pivot_row_col[0],last_col]

    const_multiplier_prefix='c'
    const_multiplier_idx=1
    x_nullspace_solutions_list=[]
    for non_pivot_col in non_pivot_cols:
        x_nullspace_solution=np.zeros(A.shape[1],dtype=float)
        x_nullspace_solution[non_pivot_col]=-1
        idx=0
        for pivot_col in pivot_cols:
            if pivot_col < non_pivot_col:
               x_nullspace_solution[pivot_col]=A[idx,non_pivot_col]
               idx+=1
        x_nullspace_solution_str=' + {}*{}'.format(const_multiplier_prefix+str(const_multiplier_idx),x_nullspace_solution)
        const_multiplier_idx+=1
        x_nullspace_solutions.append(x_nullspace_solution)
        x_nullspace_solutions_list.append(x_nullspace_solution_str)
    x_nullspace_solutions_str=' '.join(x_nullspace_solutions_list)
    x_generic_solution_str='Generic solution would be {} {}'.format(x_particular,x_nullspace_solutions_str)

    return  x_particular,x_nullspace_solutions,x_generic_solution_str



In [240]:
def create_matrix():
    A = np.array([[ 1, 0, 8 ,-4 ], [0 ,1, 2, 12]])
    A=A.astype(float)
    return A

In [259]:
A=create_matrix()
#print(A)
b=np.array([42,8])
b=b.astype(float)
x_particular,x_nullspace_solutions,x_generic_solution_str=calculate_generic_solution(A,b)
print('Particular solution is {}'.format(x_particular))
print('NullSpace solution(s) are {}'.format(x_nullspace_solutions))
print(x_generic_solution_str)



Particular solution is [42.  8.  0.  0.]
NullSpace solution(s) are [array([ 8.,  2., -1.,  0.]), array([-4., 12.,  0., -1.])]
Generic solution would be [42.  8.  0.  0.]  + c1*[ 8.  2. -1.  0.]  + c2*[-4. 12.  0. -1.]
